In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import LogisticRegression

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


## Reading Avg w2v vectors(100-D)

In [3]:
avg_w2v =pd.read_pickle("avg_w2v_vec_100")
print(avg_w2v.shape)
avg_w2v.head()

(122109, 101)


,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,0.000778,-0.000347,0.000871,-0.000326,0.000397,0.000361,0.000081,0.001098,0.000355,0.000008,...,-0.000193,0.000537,0.000594,0.000397,-0.000584,-0.001169,0.000315,-0.000140,0.000049,1
1,0.000567,0.000178,0.000499,0.000169,-0.000260,0.000105,-0.000142,0.000246,-0.000276,-0.000637,...,-0.000212,0.000260,0.000580,0.000210,0.000176,0.000737,0.000288,0.000127,0.000436,1
2,-0.000123,-0.000488,0.000094,-0.001271,-0.001240,0.000217,0.001422,-0.000463,-0.000512,0.001132,...,0.000842,0.000572,0.000398,0.000072,0.000744,0.000469,-0.000092,-0.000362,-0.000065,0
3,0.000780,-0.000327,0.000449,-0.000747,-0.000130,-0.000132,0.000667,-0.000025,-0.000042,0.000242,...,-0.000111,0.000417,0.000003,-0.000190,-0.000268,0.000645,0.000049,-0.000225,-0.000452,1
4,0.000114,0.001019,-0.000837,0.000060,-0.000303,-0.000609,-0.000855,0.000182,0.000044,0.000346,...,-0.000147,-0.000636,0.000251,0.001050,0.000173,-0.000852,-0.000106,0.000595,0.000065,1


In [5]:
x =avg_w2v.iloc[:,:100]
y =avg_w2v.iloc[:,100]

In [6]:
print(x.shape)
y.shape

(122109, 100)


(122109,)

## Column Standardization

In [3]:
from sklearn.preprocessing import StandardScaler

In [8]:
a =StandardScaler()
x =a.fit_transform(x)

In [9]:
x_train,x_test,y_train,y_test =train_test_split(x,y,test_size =0.3)

In [10]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
y_test.shape

(85476, 100)
(36633, 100)
(85476,)


(36633,)

In [11]:
tuned_params = [{'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]}]

In [12]:
model = GridSearchCV(LogisticRegression(), tuned_params, scoring = 'accuracy')
model.fit(x_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'C': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]}],
       pre_dispatch='2*n_jobs', refit=True, scoring='accuracy', verbose=0)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [14]:
print(model.best_estimator_)
print(model.score(x_test, y_test))

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
0.710889089073


In [15]:
y_pred = model.best_estimator_.predict(x_test)

In [16]:
acc = accuracy_score(y_test, y_pred, normalize=True) * float(100)
acc

71.088908907269399

## Confusion Matrix

In [17]:
confusion_matrix(y_test,y_pred).T

array([[11548,  4896],
       [ 5695, 14494]], dtype=int64)

## L1 regularization and sparsity

In [18]:
clf = LogisticRegression(C=1, penalty = 'l1')
clf.fit(x_train, y_train)
w = clf.coef_    
print(np.count_nonzero(w))

100


This means 100 out of 100 are non zero.That means for regularization takes place.

In [19]:
clf = LogisticRegression(C=0.1, penalty = 'l1')
clf.fit(x_train, y_train)
w = clf.coef_
print(np.count_nonzero(w))

99


As we can see clearly as c decreases sparsity increases.This time 99 features out of 100  are non zero,means 1 features is removed.

In [21]:
clf = LogisticRegression(C=0.001, penalty = 'l1')
clf.fit(x_train, y_train)
w = clf.coef_
print(np.count_nonzero(w))

55


Now the 55 out of 100 are non zero.That means 45 are zero which are removed.

In [23]:
clf = LogisticRegression(C=0.00025, penalty = 'l1')
clf.fit(x_train, y_train)
w = clf.coef_
print(np.count_nonzero(w))

10


At c=0.00025 we are getting top 10 features and rest becomes zero.